## Utils

In [14]:
import telnetlib 
import json

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

HOST = 'socket.cryptohack.org' 

# ONE TIME PAD

### Gotta Go Fast 

Since server uses current time as input to SHA256 as key we can send 2 challenges fast and get same key for encryption of flag and our message

In [13]:
from pwn import xor 
PORT = 13372
tn = telnetlib.Telnet(HOST, PORT)

readline()

m_prime = b'crypto{????????????????????}'
forgery = {}
forgery['option'] = 'encrypt_data'
forgery['input_data'] = m_prime.hex() 

json_send(forgery)
r = json_recv()
c_prime = bytes.fromhex(r['encrypted_data']) 
k= xor(c_prime, m_prime)

json_send({'option':'get_flag'})
r = json_recv()
c = bytes.fromhex(r['encrypted_flag'])
print(xor(c, k).decode())

crypto{t00_f4st_t00_furi0u5}


### No leaks

So it checks if ciphertext has same byte as flag, so basically if we get ciphertext back, we can remove from guesses all the characters we've seen there for our bytes. 

In [36]:
import base64 

PORT = 13370
tn = telnetlib.Telnet(HOST, PORT)

readline()
FLAG = "crypto{????????????}"

guesses = [] 
for _ in range(len(FLAG)-8): 
    guesses.append([i for i in range(33,127)])

while True: 
    progress = sum([len(i) for i in guesses]) 
    
    if progress == 12: 
        break 

    json_send({'msg': 'request'})
    r = json_recv()
    
    if "error" not in r:  
        c = base64.b64decode(r['ciphertext'])
        c = c[7:len(FLAG)-1]
        for i in range(len(c)): 
            try: 
                guesses[i].remove(c[i]) 
            except Exception: 
                continue 
f = "crypto{"           
for i, g in enumerate(guesses): 
    f+=chr(g[0]) 
print(f+"}")

crypto{unr4nd0m_07p}
